<a href="https://colab.research.google.com/github/dh1105/Sentence-Entailment/blob/main/Sentence_entailment_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-12-05 16:00:00--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.64.190
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.64.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

eNews-vectors-negat  88%[================>   ]   1.36G  95.4MB/s    eta 2s     ^C


In [ ]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

^C


In [ ]:
TRAIN_PATH = '/content/multinli_1.0/multinli_1.0_train.jsonl'
VAL_PATH   = '/content/multinli_1.0/multinli_1.0_dev_matched.jsonl'

In [ ]:
def read_jsonl_file(path):
  with open(path, 'r') as json_file:
    json_list = list(json_file)

  results = []
  for json_str in json_list:
      results.append(json.loads(json_str))
  return results

In [ ]:
mutlinli_train_df = pd.DataFrame(read_jsonl_file(TRAIN_PATH))
mutlinli_val_df  = pd.DataFrame(read_jsonl_file(VAL_PATH))

In [ ]:
train_df = mutlinli_train_df[['gold_label', 'sentence1', 'sentence2']]
val_df  = mutlinli_val_df[['gold_label', 'sentence1', 'sentence2']]

In [ ]:
train_df

,gold_label,sentence1,sentence2
0,neutral,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...
1,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...
2,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...
3,entailment,How do you know? All this is their information...,This information belongs to them.
4,neutral,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.
...,...,...,...
392697,contradiction,"Clearly, California can - and must - do better.",California cannot do any better.
392698,neutral,It was once regarded as the most beautiful str...,So many of the original buildings had been rep...
392699,entailment,Houseboats are a beautifully preserved traditi...,The tradition of houseboats originated while t...
392700,neutral,Obituaries fondly recalled his on-air debates ...,The obituaries were beautiful and written in k...


In [ ]:
val_df

,gold_label,sentence1,sentence2
0,neutral,The new rights are nice enough,Everyone really likes the newest benefits
1,contradiction,This site includes a list of all award winners...,The Government Executive articles housed on th...
2,entailment,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ..."
3,contradiction,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...
4,contradiction,i don't know um do you do a lot of camping,I know exactly.
...,...,...,...
9995,neutral,"Since 1998, LSC has initiated and overseen sig...",LSC has been focusing on improving it's state ...
9996,contradiction,Eighty percent of pagers in the United States ...,Pagers in the United States were unaffected by...
9997,entailment,"Finally, the FDA will conduct workshops, issue...",The FDA is set to conduct workshops.
9998,entailment,Cirque du Soleil's The latest from the acclaim...,Cirque du Soleil is an international troupe.


In [ ]:
train_df['gold_label'].unique()

array(['neutral', 'entailment', 'contradiction'], dtype=object)

In [ ]:
val_df['gold_label'].unique()

array(['neutral', 'contradiction', 'entailment', '-'], dtype=object)

In [ ]:
val_df = val_df[val_df['gold_label'] != '-']
val_df['gold_label'].unique()

array(['neutral', 'contradiction', 'entailment'], dtype=object)

In [ ]:
# train_df.to_csv('/content/drive/My Drive/Sentence entailment/train_data.csv')
# val_df.to_csv('/content/drive/My Drive/Sentence entailment/val_data.csv')

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d takuok/glove840b300dtxt

100% 2.08G/2.08G [00:22<00:00, 35.6MB/s]



In [ ]:
!unzip /content/glove840b300dtxt.zip

Archive:  /content/glove840b300dtxt.zip
  inflating: glove.840B.300d.txt     


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/Sentence entailment/train_data.csv')
val_df   = pd.read_csv('/content/drive/My Drive/Sentence entailment/val_data.csv')

In [ ]:
def get_word_map(count):  
  word_map = {}
  for num in count:
    if num in word_map:
      word_map[num] += 1
    else:
      word_map[num] = 1

  return word_map

In [ ]:
train_df[(train_df['sentence1'] == '') | (train_df['sentence2'] == '')]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [ ]:
train_df[(train_df['sentence1'].isnull()) | (train_df['sentence2'].isnull())]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
val_df[(val_df['sentence1'] == '') | (val_df['sentence2'] == '')]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
train_df[(train_df['sentence1'].str.split().str.len() == 1) | (train_df['sentence2'].str.split().str.len() == 1)]

,Unnamed: 0,gold_label,sentence1,sentence2
8,8,contradiction,Gays and lesbians.,Heterosexuals.
1489,1489,neutral,Really?,Is that the case?
1738,1738,entailment,The devil!,Lucifer!
1819,1819,contradiction,'Unlikely.,That is very possible.
2217,2217,contradiction,'Don't worry.,Panic!
...,...,...,...,...
390647,390647,contradiction,'Stretching?',Remaining stiff.
390717,390717,neutral,Sparkly things.,Glitter.
391451,391451,contradiction,Each year,never
392153,392153,contradiction,Rocamadour,The Arch de Triumph.


In [ ]:
count_premise = train_df['sentence1'].str.split().str.len()
get_word_map(count_premise)

In [ ]:
count_hypothesis = train_df['sentence2'].str.split().str.len()
get_word_map(count_hypothesis)

In [ ]:
val_df

,Unnamed: 0,gold_label,sentence1,sentence2
0,0,neutral,The new rights are nice enough,Everyone really likes the newest benefits
1,1,contradiction,This site includes a list of all award winners...,The Government Executive articles housed on th...
2,2,entailment,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ..."
3,3,contradiction,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...
4,4,contradiction,i don't know um do you do a lot of camping,I know exactly.
...,...,...,...,...
9810,9995,neutral,"Since 1998, LSC has initiated and overseen sig...",LSC has been focusing on improving it's state ...
9811,9996,contradiction,Eighty percent of pagers in the United States ...,Pagers in the United States were unaffected by...
9812,9997,entailment,"Finally, the FDA will conduct workshops, issue...",The FDA is set to conduct workshops.
9813,9998,entailment,Cirque du Soleil's The latest from the acclaim...,Cirque du Soleil is an international troupe.


In [ ]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def clean_text(text, stop_words = False, lemmatization = False):
  text = text.lower().split()
  if stop_words:
    stop = stopwords.words('english')
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
  
  text = " ".join(text)
  text = re.sub("[^A-Za-z']+", ' ', str(text)).replace("'", '')
  text = re.sub(r"\bum*\b", "", text)
  text = re.sub(r"\buh*\b", "", text)
  text = re.sub(r"won\'t", "will not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  if lemmatization:
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text)])
  
  text = text.translate(str.maketrans('', '', punctuation))
  return text.strip()

In [ ]:
train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [ ]:
train_df[(train_df['sentence1'] == '') | (train_df['sentence2'] == '')]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
val_df[(val_df['sentence1'] == '') | (val_df['sentence2'] == '')]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [ ]:
train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]

In [ ]:
train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]

In [ ]:
val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)]

,Unnamed: 0,gold_label,sentence1,sentence2


In [ ]:
train_df

,Unnamed: 0,gold_label,sentence1,sentence2
0,0,neutral,conceptually cream skimming ha two basic dimen...,product and geography are what make cream skim...
1,1,entailment,you know during the season and i guess at at y...,you lose the thing to the following level if t...
2,2,entailment,one of our number will carry out your instruct...,a member of my team will execute your order wi...
3,3,entailment,how do you know all this is their information ...,this information belongs to them
4,4,neutral,yeah i tell you what though if you go price so...,the tennis shoe have a range of price
...,...,...,...,...
392697,392697,contradiction,clearly california can and must do better,california can not do any better
392698,392698,neutral,it wa once regarded a the most beautiful stree...,so many of the original building had been repl...
392699,392699,entailment,houseboat are a beautifully preserved traditio...,the tradition of houseboat originated while th...
392700,392700,neutral,obituary fondly recalled his on air debate and...,the obituary were beautiful and written in kin...


In [ ]:
val_df

,Unnamed: 0,gold_label,sentence1,sentence2
0,0,neutral,the new right are nice enough,everyone really like the newest benefit
1,1,contradiction,this site includes a list of all award winner ...,the government executive article housed on the...
2,2,entailment,i dont know i i have mixed emotion about him s...,i like him for the most part but would still e...
3,3,contradiction,yeah i i think my favorite restaurant is alway...,my favorite restaurant are always at least a h...
4,4,contradiction,i dont know do you do a lot of camping,i know exactly
...,...,...,...,...
9810,9995,neutral,since lsc ha initiated and overseen significan...,lsc ha been focusing on improving it state del...
9811,9996,contradiction,eighty percent of pager in the united state we...,pager in the united state were unaffected by t...
9812,9997,entailment,finally the fda will conduct workshop issue gu...,the fda is set to conduct workshop
9813,9998,entailment,cirque du soleils the latest from the acclaime...,cirque du soleil is an international troupe


In [ ]:
val_df

,Unnamed: 0,gold_label,sentence1,sentence2
0,0,neutral,the new right are nice enough,everyone really like the newest benefit
1,1,contradiction,this site includes a list of all award winner ...,the government executive article housed on the...
2,2,entailment,i dont know i i have mixed emotion about him s...,i like him for the most part but would still e...
3,3,contradiction,yeah i i think my favorite restaurant is alway...,my favorite restaurant are always at least a h...
4,4,contradiction,i dont know do you do a lot of camping,i know exactly
...,...,...,...,...
9810,9995,neutral,since lsc ha initiated and overseen significan...,lsc ha been focusing on improving it state del...
9811,9996,contradiction,eighty percent of pager in the united state we...,pager in the united state were unaffected by t...
9812,9997,entailment,finally the fda will conduct workshop issue gu...,the fda is set to conduct workshop
9813,9998,entailment,cirque du soleils the latest from the acclaime...,cirque du soleil is an international troupe


In [ ]:
train_val_df = pd.concat([train_df, val_df])

In [ ]:
def pair_generator(df):
  sentence_pair  = []
  sentence_label = []
  for _, row in df.iterrows():
    sentence_pair.append((row['sentence1'], row['sentence2']))
    sentence_label.append(row['gold_label'])
  return sentence_pair, sentence_label

In [ ]:
sentence_pairs, _ = pair_generator(train_val_df)
train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

In [ ]:
labels = set(train_sentence_labels)
print(labels)

{'contradiction', 'entailment', 'neutral'}


In [ ]:
tag2idx = {word: i for i, word in enumerate(labels)}
print(tag2idx)

{'contradiction': 0, 'entailment': 1, 'neutral': 2}


In [ ]:
train_labels = [tag2idx[t] for t in train_sentence_labels]
val_labels = [tag2idx[t] for t in val_sentence_labels]

In [ ]:
class Vocabulary:
  def __init__(self):
    self.word2index = {}
    self.word2count = {}
    self.index2word = {}
    self.n_words = 0

  def addSentence(self, sentence):
    for word in word_tokenize(sentence):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words + 1
      self.word2count[word] = 1
      self.index2word[self.n_words + 1] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [ ]:
class DataSetLoader(Dataset):
  def __init__(self, sentence_pair, labels):
    self.sentence_pair = sentence_pair
    self.labels        = labels

  def __len__(self):
    return len(self.sentence_pair)

  def __getitem__(self, index):
    return self.sentence_pair[index], self.labels[index]

In [ ]:
vocab = Vocabulary()
for data in [sentence_pairs]:
  for sentence_pair in data:
    premise    = sentence_pair[0]
    hypothesis = sentence_pair[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

print("Vocab size:", len(vocab.word2index))

Vocab size: 71154


In [ ]:
def get_pair_indices(sentence_pairs):
  indices_pairs = []
  for sentence_pair in sentence_pairs:
    premise = sentence_pair[0]
    premise_indices = [vocab.word2index[w] for w in word_tokenize(premise)]
    hypothesis = sentence_pair[1]
    hypothesis_indices = [vocab.word2index[w] for w in word_tokenize(hypothesis)]
    indices_pairs.append((premise_indices, hypothesis_indices))
  return indices_pairs

In [ ]:
train_data = DataSetLoader(get_pair_indices(train_sentence_pairs), train_labels)
val_data   = DataSetLoader(get_pair_indices(val_sentence_pairs), val_labels)

In [ ]:
EPOCHS = 25
BATCH_SIZE = 32
EMBEDDING_SIZE = 300
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 64
LEARNING_RATE = 0.005
STACKED_LAYERS = 2
EMBEDDING_PATH = '/content/GoogleNews-vectors-negative300.bin'
GLOVE_EMBEDDING = '/content/glove.840B.300d.txt'

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

12271 307


In [ ]:
def get_coefs(word, *arr):
  return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
  with open(path) as f:
    return dict(get_coefs(*line.strip().split(' ')) for line in f)

embeddings_index = load_embeddings(GLOVE_EMBEDDING)

In [ ]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

Embedded count: 58685


(71155, 300)

In [ ]:
# word2vec_model = KeyedVectors.load_word2vec_format(EMBEDDING_PATH, binary=True)

# weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
# for word, lang_word_index in vocab.word2index.items():
#   if word in word2vec_model.vocab:
#     weights[lang_word_index] = word2vec_model.word_vec(word)

# del word2vec_model
# weights.shape

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
class LSTM(nn.Module):
  def __init__(self, vocab_size, hidden_size, target_size, stacked_layers, weights_matrix, bidirectional):
    super(LSTM, self).__init__()
    self.vocab_size     = vocab_size
    self.hidden_size    = hidden_size
    self.bidirectional  = bidirectional
    self.target_size    = target_size
    self.stacked_layers = stacked_layers
    num_embeddings, embedding_dim = weights_matrix.shape[0], weights_matrix.shape[1]
    self.embedding = nn.Embedding(num_embeddings, embedding_dim)
    self.embedding.weight.data.copy_(torch.from_numpy(weights_matrix))
    self.embedding.weight.requires_grad = True

    self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_size, num_layers=self.stacked_layers, batch_first = True, dropout=0.2, bidirectional=bidirectional)
    
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p = 0.2)

    self.FC_concat1 = nn.Linear(2 * 2 * hidden_size if bidirectional else 2 * hidden_size, 128)
    self.FC_concat2 = nn.Linear(128, 64)
    self.FC_concat3 = nn.Linear(64, 32)

    for lin in [self.FC_concat1, self.FC_concat2]:
		    nn.init.xavier_uniform_(lin.weight)
		    nn.init.zeros_(lin.bias)

    self.output = nn.Linear(32, self.target_size)

    self.out = nn.Sequential(
			self.FC_concat1,
			self.relu,
			self.dropout,
			self.FC_concat2,
			self.relu,
      self.FC_concat3,
      self.relu,
			self.dropout,
			self.output
		)

  def forward_once(self, seq, hidden, seq_len):
    embedd_seq = self.embedding(seq)
    packed_seq = pack_padded_sequence(embedd_seq, lengths=seq_len, batch_first=True, enforce_sorted=False)
    output, (hidden, _) = self.lstm(packed_seq, hidden)
    return hidden

  def forward(self, input, premise_len, hypothesis_len):
    premise    = input[0]
    hypothesis = input[1]
    batch_size = premise.size(0)

    h0 = torch.zeros(self.stacked_layers*2 if self.bidirectional else self.stacked_layers, batch_size, self.hidden_size).to(device) # 2 for bidirection 
    c0 = torch.zeros(self.stacked_layers*2 if self.bidirectional else self.stacked_layers, batch_size, self.hidden_size).to(device)

    # hidden = self.init_hidden(batch_size)

    premise    = self.forward_once(premise, (h0, c0), premise_len)
    hypothesis = self.forward_once(hypothesis, (h0, c0), hypothesis_len)
    
    combined_outputs  = torch.cat((premise, hypothesis, torch.abs(premise - hypothesis), premise * hypothesis), dim=2)

    return self.out(combined_outputs[-1])

  # def init_hidden(self, batch_size):
  #   weight = next(self.parameters()).data
  #   hidden = (weight.new(self.stacked_layers, batch_size, self.hidden_size).zero_().to(device),
  #                     weight.new(self.stacked_layers, batch_size, self.hidden_size).zero_().to(device))
  #   return hidden

In [ ]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
import time

def train(model, train_loader, val_loader, criterion, optimizer):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for val in train_loader:
      sentence_pairs, labels = map(list, zip(*val))

      premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in sentence_pairs]
      hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in sentence_pairs]
      batch = len(premise_seq)

      premise_len    = list(map(len, premise_seq))
      hypothesis_len = list(map(len, hypothesis_seq))

      temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
      premise_seq    = temp[:batch, :]
      hypothesis_seq = temp[batch:, :]
      labels         = torch.tensor(labels).long().to(device)

      model.zero_grad()
      prediction = model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)

      loss = criterion(prediction, labels)
      acc  = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for val in val_loader:
        sentence_pairs, labels = map(list, zip(*val))

        premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in sentence_pairs]
        hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in sentence_pairs]
        batch = len(premise_seq)

        premise_len    = list(map(len, premise_seq))
        hypothesis_len = list(map(len, hypothesis_seq))

        temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
        premise_seq    = temp[:batch, :]
        hypothesis_seq = temp[batch:, :]

        premise_seq    = premise_seq.to(device)
        hypothesis_seq = hypothesis_seq.to(device)
        labels         = torch.tensor(labels).long().to(device)

        model.zero_grad()
        prediction = model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
        
        loss = criterion(prediction, labels)
        acc  = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)

    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    torch.cuda.empty_cache()

In [ ]:
lstm_model = LSTM(VOCAB_SIZE, HIDDEN_SIZE, TARGET_SIZE, STACKED_LAYERS, weights, True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

LSTM(
  (embedding): Embedding(71155, 300)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=256, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [ ]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

00:08:01.14
Epoch 1: train_loss: 0.9631 train_acc: 0.5296 | val_loss: 0.9337 val_acc: 0.5553
00:08:01.35
Epoch 2: train_loss: 0.9231 train_acc: 0.5660 | val_loss: 0.9151 val_acc: 0.5830
00:08:02.74
Epoch 3: train_loss: 0.9069 train_acc: 0.5789 | val_loss: 0.8849 val_acc: 0.5913
00:08:03.71
Epoch 4: train_loss: 0.8986 train_acc: 0.5845 | val_loss: 0.8913 val_acc: 0.5958
00:08:04.78
Epoch 5: train_loss: 0.8959 train_acc: 0.5868 | val_loss: 0.8968 val_acc: 0.5920
00:08:06.11
Epoch 6: train_loss: 0.8921 train_acc: 0.5881 | val_loss: 0.8749 val_acc: 0.5977
00:08:07.80
Epoch 7: train_loss: 0.8881 train_acc: 0.5913 | val_loss: 0.8820 val_acc: 0.5932
00:08:08.89
Epoch 8: train_loss: 0.8847 train_acc: 0.5942 | val_loss: 0.8731 val_acc: 0.6009
00:08:07.56
Epoch 9: train_loss: 0.8837 train_acc: 0.5944 | val_loss: 0.8868 val_acc: 0.5934
00:08:05.49
Epoch 10: train_loss: 0.8804 train_acc: 0.5966 | val_loss: 0.8702 val_acc: 0.5967
00:07:57.75
Epoch 11: train_loss: 0.8800 train_acc: 0.5987 | val_loss